In [1]:
import math as m
import numpy as np

from galpy.util import bovy_coords as b_c

In [2]:
#constants (same as used in galpy)

epoch_i=2000

if(epoch_i==2000):
    ra_NGP=m.radians(192.8594812065348)
    dec_NGP=m.radians(27.12825118085622)
    theta0=m.radians(122.9319185680026)
elif(epoch_i==1950):
    ra_NGP=m.radians(192.5)
    dec_NGP=m.radians(27.4)
    theta0=m.radians(123.)

In [3]:
#print T matrix

T00=-m.cos(theta0)*m.sin(dec_NGP)*m.cos(ra_NGP)-m.sin(theta0)*m.sin(ra_NGP)
T01=-m.cos(theta0)*m.sin(dec_NGP)*m.sin(ra_NGP)+m.sin(theta0)*m.cos(ra_NGP)
T02=m.cos(theta0)*m.cos(dec_NGP)

T10=-m.sin(theta0)*m.sin(dec_NGP)*m.cos(ra_NGP)+m.cos(theta0)*m.sin(ra_NGP)
T11=-m.sin(theta0)*m.sin(dec_NGP)*m.sin(ra_NGP)-m.cos(theta0)*m.cos(ra_NGP)
T12=m.sin(theta0)*m.cos(dec_NGP)

T20=m.cos(dec_NGP)*m.cos(ra_NGP)
T21=m.cos(dec_NGP)*m.sin(ra_NGP)
T22=m.sin(dec_NGP)

T=np.array([[T00,T01,T02],[T10,T11,T12],[T20,T21,T22]])
    
print T

[[-0.05487554 -0.8734371  -0.48383499]
 [ 0.49410945 -0.44482959  0.74698225]
 [-0.86766614 -0.19807639  0.45598379]]


In [4]:
#sample star
ra=65.984001
ra=m.radians(ra)
dec=-1.2241944
dec=m.radians(dec)
dist=5.439109129
rv=-99.2
pmra=0.27
pmdec=-1.91

In [5]:
#solve xyz

#equatorial
#right-handed
x_equ=m.cos(dec)*m.cos(ra)*dist
y_equ=m.cos(dec)*m.sin(ra)*dist
z_equ=m.sin(dec)*dist

print 'equatorial coord syst (RH)'
print 'x,y,z = {}, {}, {} (kpc)'.format(x_equ,y_equ,z_equ)

#transform to Galactic
x_gal,y_gal,z_gal=T.dot(np.array([[x_equ],[y_equ],[z_equ]]))

print '\nGalactic coord syst (RH)'
print 'x,y,z = {}, {}, {} (kpc)'.format(x_gal,y_gal,z_gal)

#transform to Galactocentric (LH) (x swaps direction, plus take Xsun into account)
x_gc=-x_gal+8.0
y_gc=y_gal
z_gc=z_gal
print '\nGalactocentric coord syst (LH)'
print 'x,y,z = {}, {}, {} (kpc)'.format(x_gc,y_gc,z_gc)

#galpy
l_GPY, b_GPY = b_c.radec_to_lb(ra, dec, degree=False, epoch=epoch_i)
x_gal_GPY, y_gal_GPY, z_gal_GPY = b_c.lbd_to_XYZ(l_GPY, b_GPY, dist, degree=False)

print '\nusing galpy:\n'
print 'Galactic coord syst (RH)'
print 'x,y,z = {}, {}, {} (kpc)'.format(x_gal_GPY,y_gal_GPY,z_gal_GPY)

x_gc_GPY,y_gc_GPY,z_gc_GPY = b_c.XYZ_to_galcenrect(x_gal_GPY,y_gal_GPY,z_gal_GPY,Xsun=8.0,Zsun=0.0,)
                                                             
print '\nGalactocentric coord syst (LH)'
print 'x,y,z = {}, {}, {} (kpc)'.format(x_gc_GPY,y_gc_GPY,z_gc_GPY)

equatorial coord syst (RH)
x,y,z = 2.21316711805, 4.96712149717, -0.116204376391 (kpc)

Galactic coord syst (RH)
x,y,z = [-4.40369322], [-1.20277845], [-2.95714697] (kpc)

Galactocentric coord syst (LH)
x,y,z = [12.40369322], [-1.20277845], [-2.95714697] (kpc)

using galpy:

Galactic coord syst (RH)
x,y,z = -4.4036932152, -1.20277845133, -2.95714696635 (kpc)

Galactocentric coord syst (LH)
x,y,z = 12.4036901013, -1.20277001142, -2.95715503626 (kpc)


In [6]:
A00=m.cos(ra)*m.cos(dec)
A01=-m.sin(ra)
A02=-m.cos(ra)*m.sin(dec)

A10=m.sin(ra)*m.cos(dec)
A11=m.cos(ra)
A12=-m.sin(ra)*m.sin(dec)

A20=m.sin(dec)
A21=0.
A22=m.cos(dec)

A=np.array([[A00,A01,A02],[A10,A11,A12],[A20,A21,A22]])

k=4.74
v_obs=np.array([[rv],[k*dist*pmra],[k*dist*pmdec]])
v_equ=A.dot(v_obs)
v_gal=T.dot(v_equ)

print 'Galactic coord sys (RH):'
print 'v = {}, {}, {} (km/s)'.format(v_gal[0],v_gal[1],v_gal[2])

vx_gc=-v_gal[0] #x-component swaps signs (RH->LH)
vy_gc=v_gal[1]+220. #add LSR mot
vz_gc=v_gal[2]

print '\nGalactocentric coord sys (LH):'
print 'v = {}, {}, {} (km/s)'.format(vx_gc,vy_gc,vz_gc)

print '\nusing galpy:\n'
pmll_GPY,pmbb_GPY=b_c.pmrapmdec_to_pmllpmbb(pmra,pmdec,ra,dec,degree=False,epoch=epoch_i)

vx_GPY,vy_GPY,vz_GPY=b_c.vrpmllpmbb_to_vxvyvz(rv,pmll_GPY,pmbb_GPY,l_GPY,b_GPY,dist,degree=False)
print 'Galactic coord sys (RH):'
print 'v = {}, {}, {} (km/s)'.format(vx_GPY,vy_GPY,vz_GPY)

vx_gc_GPY,vy_gc_GPY,vz_gc_GPY=b_c.vxvyvz_to_galcenrect(vx_GPY,vy_GPY,vz_GPY,vsun=[0.,220.,0.],Xsun=8.0,Zsun=0.0)
print '\nGalactocentric coord sys (LH):'
print 'v = {}, {}, {} (km/s)'.format(vx_gc_GPY,vy_gc_GPY,vz_gc_GPY)

Galactic coord sys (RH):
v = [102.87283202], [-19.02427114], [37.00229236] (km/s)

Galactocentric coord sys (LH):
v = [-102.87283202], [200.97572886], [37.00229236] (km/s)

using galpy:

Galactic coord sys (RH):
v = 102.875070891, -19.0283366704, 37.0006118931 (km/s)

Galactocentric coord sys (LH):
v = -102.875000229, 200.971513286, 37.0007311953 (km/s)


In [8]:
#cylindrical

vr=(x_gc*vx_gc+y_gc*vy_gc)/(m.sqrt(x_gc**2+y_gc**2))

vphi=(x_gc*vy_gc-y_gc*vx_gc)/(x_gc**2+y_gc**2)
vphi=vphi*(m.sqrt(x_gc**2+y_gc**2))

print 'vr = {}'.format(vr)
print 'vphi = {}'.format(vphi)


print '\nusing galpy:\n'

vr_GPY,vphi_GPY,vz_GPY=b_c.vxvyvz_to_galcencyl(vx_GPY,vy_GPY,vz_GPY,x_gc_GPY,y_gc_GPY,z_gc_GPY,vsun=[0.,220.,0.],Xsun=8.0,Zsun=0.0)
print 'v_cyl = {}, {}, {} (km/s)'.format(vr_GPY,vphi_GPY,vz_GPY)

print '\nalternative galpy:\n'
print b_c.rect_to_cyl_vec(vx_gc_GPY,vy_gc_GPY,vz_gc_GPY,x_gc_GPY,y_gc_GPY,z_gc_GPY)


vr = [-121.79006408]
vphi = [190.10850441]

using galpy:

v_cyl = -121.791691728, 190.104178414, 37.0008504975 (km/s)

alternative galpy:
(-121.79169172855597, 190.104178413755, 37.00073119533055)
